In [7]:
pip install pandas numpy fuzzywuzzy python-Levenshtein


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



   ---------------------------------------- 0.0/98.8 kB ? eta -:--:--
   ---- ----------------------------------- 10.2/98.8 kB ? eta -:--:--
   ------------ --------------------------- 30.7/98.8 kB 435.7 kB/s eta 0:00:01
   ----------------------------- ---------- 71.7/98.8 kB 787.7 kB/s eta 0:00:01
   ---------------------------------------- 98.8/98.8 kB 942.1 kB/s eta 0:00:00
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ----- ---------------------------------- 0.2/1.6 MB 6.1 MB/s eta 0:00:01
   ----------- ---------------------------- 0.5/1.6 MB 5.8 MB/s eta 0:00:01
   -------------------------- ------------- 1.1/1.6 MB 8.5 MB/s eta 0:00:01
   ---------------------------------------- 1.6/1.6 MB 10.2 MB/s eta 0:00:00


In [8]:
import pandas as pd
import numpy as np
import re
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [4]:
facebook_data = pd.read_csv(r'C:\Users\Fabius S Pop\Desktop\JrDE_Assign\facebook_dataset.csv', delimiter=',', on_bad_lines='skip')
website_data = pd.read_csv(r'C:\Users\Fabius S Pop\Desktop\JrDE_Assign\website_dataset.csv', delimiter=';', on_bad_lines='skip')
google_data = pd.read_csv(r'C:\Users\Fabius S Pop\Desktop\JrDE_Assign\google_dataset.csv', delimiter=',', on_bad_lines='skip')

C:\Users\Fabius S Pop\AppData\Local\Temp\ipykernel_64844\3558242565.py:3: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  google_data = pd.read_csv(r'C:\Users\Fabius S Pop\Desktop\JrDE_Assign\google_dataset.csv', delimiter=',', on_bad_lines='skip')


In [9]:
facebook_data.head

<bound method NDFrame.head of                            domain  \
0              euro-hygiene-34.fr   
1         lakesidehomeservices.ca   
2               rossiterboats.com   
3            caa-architecture.com   
4                   apexsurety.ca   
...                           ...   
71162                   cadex.com   
71163        coastlinecatering.ca   
71164                    hcwd.com   
71165  serenityskincarestudio.com   
71166     easttorontofootcare.com   

                                                 address  \
0      134 rue entrepreneurs, za du vigné, 30420, cal...   
1                                                    NaN   
2                                                    NaN   
3      601 cordova st w # 270, v6b 1g1, vancouver, bc...   
4      unit 3 - 4 donald street, r3l 2t7, winnipeg, m...   
...                                                  ...   
71162  22000 fraserwood way, v6w1j6, richmond, bc, ca...   
71163  351 prince edward drive north, m8x 2l4

In [10]:
website_data.head

<bound method NDFrame.head of                       root_domain domain_suffix language  \
0                converterman.com           NaN      NaN   
1             clothesencounter.ca            ca       en   
2                  investa.com.au        com.au       en   
3               timminsgarage.com           com       en   
4                    destinate.ca            ca       en   
...                           ...           ...      ...   
72013                agistudio.ca            ca       en   
72014                    le279.ca            ca       fr   
72015  absolutedatadestruction.ca            ca       en   
72016                     hhms.ca            ca       en   
72017       studioaeinteriors.com           com       en   

                                       legal_name       main_city  \
0                                             NaN             NaN   
1                                             NaN        cardigan   
2      Investa Wholesale Funds Management 

In [11]:
google_data.head

<bound method NDFrame.head of                                                   address  \
0       28 Central Coast Hwy, West Gosford NSW 2250, A...   
1        400 Scott St, St. Catharines, ON L2M 3W2, Canada   
2           191 Pleasant St, Yarmouth, NS B5A 2J9, Canada   
3       11040 Santa Monica Blvd Suite 370, Los Angeles...   
4                                                     NaN   
...                                                   ...   
346920  12b Marsh Cl, Winterbourne, Bristol BS36 1HY, ...   
346921                                                NaN   
346922                                                NaN   
346923                                                NaN   
346924                   8024 22nd Ave, Kenosha, WI 53143   

                                 category            city country_code  \
0                 Fabric-Based Home Goods         gosford           au   
1                             Book Stores  st. catharines           ca   
2       Other B

In [12]:
facebook_data.shape

(71167, 16)

In [13]:
website_data.shape

(72018, 11)

In [14]:
google_data.shape

(346925, 15)

In [16]:
facebook_data.columns.tolist()

['domain',
 'address',
 'categories',
 'city',
 'country_code',
 'country_name',
 'description',
 'email',
 'link',
 'name',
 'page_type',
 'phone',
 'phone_country_code',
 'region_code',
 'region_name',
 'zip_code']

In [17]:
website_data.columns.tolist()

['root_domain',
 'domain_suffix',
 'language',
 'legal_name',
 'main_city',
 'main_country',
 'main_region',
 'phone',
 'site_name',
 'tld',
 's_category']

In [18]:
google_data.columns.tolist()

['address',
 'category',
 'city',
 'country_code',
 'country_name',
 'name',
 'phone',
 'phone_country_code',
 'raw_address',
 'raw_phone',
 'region_code',
 'region_name',
 'text',
 'zip_code',
 'domain']

In [19]:
facebook_data.isnull().sum()

domain                    0
address               14477
categories            16574
city                  26600
country_code          13976
country_name          26302
description           43978
email                 51414
link                      0
name                      0
page_type                 4
phone                 26818
phone_country_code    33616
region_code           26605
region_name           26605
zip_code              35544
dtype: int64

In [27]:
fb_total_records = facebook_data.shape[0]
fb_total_records

71167

In [28]:
fb_missing_values = facebook_data.isnull().sum()
fb_missing_percent = (fb_missing_values / fb_total_records) * 100
fb_missing_percent

domain                 0.000000
address               20.342293
categories            23.288884
city                  37.376874
country_code          19.638316
country_name          36.958141
description           61.795495
email                 72.244158
link                   0.000000
name                   0.000000
page_type              0.005621
phone                 37.683196
phone_country_code    47.235376
region_code           37.383900
region_name           37.383900
zip_code              49.944497
dtype: float64

In [20]:
website_data.isnull().sum()

root_domain          1
domain_suffix      118
language          5192
legal_name       40029
main_city        11052
main_country      7149
main_region      11085
phone             6378
site_name         3304
tld                261
s_category        1384
dtype: int64

In [29]:
web_total_records = website_data.shape[0]
web_total_records

72018

In [30]:
web_missing_values = website_data.isnull().sum()
web_missing_percent = (web_missing_values / web_total_records) * 100
web_missing_percent

root_domain       0.001389
domain_suffix     0.163848
language          7.209309
legal_name       55.581938
main_city        15.346163
main_country      9.926685
main_region      15.391985
phone             8.856119
site_name         4.587742
tld               0.362409
s_category        1.921742
dtype: float64

In [21]:
google_data.isnull().sum()

address                25610
category               51063
city                   44726
country_code           51564
country_name           44690
name                      32
phone                  32136
phone_country_code    112821
raw_address            44016
raw_phone              27948
region_code            44876
region_name            44871
text                    3514
zip_code               81279
domain                     2
dtype: int64

In [31]:
goog_total_records = google_data.shape[0]
goog_total_records

346925

In [32]:
goog_missing_values = google_data.isnull().sum()
goog_missing_percent = (goog_missing_values / goog_total_records) * 100
goog_missing_percent

address                7.381999
category              14.718743
city                  12.892124
country_code          14.863155
country_name          12.881747
name                   0.009224
phone                  9.263097
phone_country_code    32.520285
raw_address           12.687468
raw_phone              8.055920
region_code           12.935361
region_name           12.933919
text                   1.012899
zip_code              23.428407
domain                 0.000576
dtype: float64

In [33]:
def reconstruct_domain(row):
    domain_parts = [str(row['root_domain']), str(row['domain_suffix']), str(row['tld'])]
    domain_parts = [part for part in domain_parts if part.lower() != 'nan']
    return ''.join(domain_parts)

In [34]:
# creating a domain column & applying clear formatting
website_data['domain'] = website_data.apply(reconstruct_domain, axis=1)
website_data['domain'] = website_data['domain'].str.lower().str.strip()

In [70]:
# standardizing domain fields
def clean_domain(domain):
    if pd.isnull(domain):
        return np.nan
    domain = domain.lower().strip()
    domain = domain.replace('www.', '')
    return domain

In [71]:
facebook_data['domain'] = facebook_data['domain'].apply(clean_domain)

In [72]:
google_data['domain'] = google_data['domain'].apply(clean_domain)

In [73]:
# standardizing company names
def clean_company_name(name):
    if pd.isnull(name):
        return np.nan
    name = name.lower().strip()
    
    # legal suffixes
    name = re.sub(r'\b(inc|llc|ltd|corp|co|inc\.|llc\.|ltd\.|corp\.|co\.)\b', '', name)
    
    # special characters
    name = re.sub(r'[^\w\s]', '', name)
    
    # extra whitespaces
    name = re.sub(r'\s+', ' ', name)
    
    return name.strip()

In [67]:
facebook_data['clean_name'] = facebook_data['name'].apply(clean_company_name)

In [74]:
facebook_data.head

<bound method NDFrame.head of                            domain  \
0              euro-hygiene-34.fr   
1         lakesidehomeservices.ca   
2               rossiterboats.com   
3            caa-architecture.com   
4                   apexsurety.ca   
...                           ...   
71162                   cadex.com   
71163        coastlinecatering.ca   
71164                    hcwd.com   
71165  serenityskincarestudio.com   
71166     easttorontofootcare.com   

                                                 address  \
0      134 rue entrepreneurs, za du vigné, 30420, cal...   
1                                                    NaN   
2                                                    NaN   
3      601 cordova st w # 270, v6b 1g1, vancouver, bc...   
4      unit 3 - 4 donald street, r3l 2t7, winnipeg, m...   
...                                                  ...   
71162  22000 fraserwood way, v6w1j6, richmond, bc, ca...   
71163  351 prince edward drive north, m8x 2l4

In [68]:
google_data['clean_name'] = google_data['name'].apply(clean_company_name)

In [75]:
google_data.head

<bound method NDFrame.head of                                                   address  \
0       28 Central Coast Hwy, West Gosford NSW 2250, A...   
1        400 Scott St, St. Catharines, ON L2M 3W2, Canada   
2           191 Pleasant St, Yarmouth, NS B5A 2J9, Canada   
3       11040 Santa Monica Blvd Suite 370, Los Angeles...   
4                                                     NaN   
...                                                   ...   
346920  12b Marsh Cl, Winterbourne, Bristol BS36 1HY, ...   
346921                                                NaN   
346922                                                NaN   
346923                                                NaN   
346924                   8024 22nd Ave, Kenosha, WI 53143   

                                 category            city country_code  \
0                 Fabric-Based Home Goods         gosford           au   
1                             Book Stores  st. catharines           ca   
2       Other B

In [69]:
website_data['clean_legal_name'] = website_data['legal_name'].apply(clean_company_name)
website_data['clean_site_name'] = website_data['site_name'].apply(clean_company_name)

In [76]:
website_data.head

<bound method NDFrame.head of                       root_domain domain_suffix language  \
0                converterman.com           NaN      NaN   
1             clothesencounter.ca            ca       en   
2                  investa.com.au        com.au       en   
3               timminsgarage.com           com       en   
4                    destinate.ca            ca       en   
...                           ...           ...      ...   
72013                agistudio.ca            ca       en   
72014                    le279.ca            ca       fr   
72015  absolutedatadestruction.ca            ca       en   
72016                     hhms.ca            ca       en   
72017       studioaeinteriors.com           com       en   

                                       legal_name       main_city  \
0                                             NaN             NaN   
1                                             NaN        cardigan   
2      Investa Wholesale Funds Management 

In [84]:
# standardizing phone numbers
def clean_phone_number(phone):
    if pd.isnull(phone):
        return np.nan
        
    # non-numeric characters
    phone = re.sub(r'\D', '', str(phone))
    return phone

In [85]:
facebook_data['clean_phone'] = facebook_data['phone'].apply(clean_phone_number)

In [86]:
facebook_data.head

<bound method NDFrame.head of                            domain  \
0              euro-hygiene-34.fr   
1         lakesidehomeservices.ca   
2               rossiterboats.com   
3            caa-architecture.com   
4                   apexsurety.ca   
...                           ...   
71162                   cadex.com   
71163        coastlinecatering.ca   
71164                    hcwd.com   
71165  serenityskincarestudio.com   
71166     easttorontofootcare.com   

                                                 address  \
0      134 rue entrepreneurs, za du vigné, 30420, cal...   
1                                                    NaN   
2                                                    NaN   
3      601 cordova st w # 270, v6b 1g1, vancouver, bc...   
4      unit 3 - 4 donald street, r3l 2t7, winnipeg, m...   
...                                                  ...   
71162  22000 fraserwood way, v6w1j6, richmond, bc, ca...   
71163  351 prince edward drive north, m8x 2l4

In [87]:
google_data['clean_phone'] = google_data['phone'].apply(clean_phone_number)

In [88]:
google_data.head

<bound method NDFrame.head of                                                   address  \
0       28 Central Coast Hwy, West Gosford NSW 2250, A...   
1        400 Scott St, St. Catharines, ON L2M 3W2, Canada   
2           191 Pleasant St, Yarmouth, NS B5A 2J9, Canada   
3       11040 Santa Monica Blvd Suite 370, Los Angeles...   
4                                                     NaN   
...                                                   ...   
346920  12b Marsh Cl, Winterbourne, Bristol BS36 1HY, ...   
346921                                                NaN   
346922                                                NaN   
346923                                                NaN   
346924                   8024 22nd Ave, Kenosha, WI 53143   

                                 category            city country_code  \
0                 Fabric-Based Home Goods         gosford           au   
1                             Book Stores  st. catharines           ca   
2       Other B

In [89]:
website_data['clean_phone'] = website_data['phone'].apply(clean_phone_number)

In [90]:
website_data.head

<bound method NDFrame.head of                       root_domain domain_suffix language  \
0                converterman.com           NaN      NaN   
1             clothesencounter.ca            ca       en   
2                  investa.com.au        com.au       en   
3               timminsgarage.com           com       en   
4                    destinate.ca            ca       en   
...                           ...           ...      ...   
72013                agistudio.ca            ca       en   
72014                    le279.ca            ca       fr   
72015  absolutedatadestruction.ca            ca       en   
72016                     hhms.ca            ca       en   
72017       studioaeinteriors.com           com       en   

                                       legal_name       main_city  \
0                                             NaN             NaN   
1                                             NaN        cardigan   
2      Investa Wholesale Funds Management 

In [95]:
# standardizing address components
def clean_address_component(component):
    if pd.isnull(component):
        return np.nan
    component = component.lower().strip()
    
    # special characters
    component = re.sub(r'[^\w\s]', '', component)
    
    # extra whitespace
    component = re.sub(r'\s+', ' ', component)
    
    return component

In [92]:
facebook_data['clean_city'] = facebook_data['city'].apply(clean_address_component)
facebook_data['clean_region'] = facebook_data['region_name'].apply(clean_address_component)
facebook_data['clean_country'] = facebook_data['country_code'].apply(clean_address_component)

In [96]:
facebook_data.head

<bound method NDFrame.head of                            domain  \
0              euro-hygiene-34.fr   
1         lakesidehomeservices.ca   
2               rossiterboats.com   
3            caa-architecture.com   
4                   apexsurety.ca   
...                           ...   
71162                   cadex.com   
71163        coastlinecatering.ca   
71164                    hcwd.com   
71165  serenityskincarestudio.com   
71166     easttorontofootcare.com   

                                                 address  \
0      134 rue entrepreneurs, za du vigné, 30420, cal...   
1                                                    NaN   
2                                                    NaN   
3      601 cordova st w # 270, v6b 1g1, vancouver, bc...   
4      unit 3 - 4 donald street, r3l 2t7, winnipeg, m...   
...                                                  ...   
71162  22000 fraserwood way, v6w1j6, richmond, bc, ca...   
71163  351 prince edward drive north, m8x 2l4

In [97]:
google_data['clean_city'] = google_data['region_name'].apply(clean_address_component)
google_data['clean_region'] = google_data['region_code'].apply(clean_address_component)
google_data['clean_country'] = google_data['country_code'].apply(clean_address_component)

In [98]:
google_data.head

<bound method NDFrame.head of                                                   address  \
0       28 Central Coast Hwy, West Gosford NSW 2250, A...   
1        400 Scott St, St. Catharines, ON L2M 3W2, Canada   
2           191 Pleasant St, Yarmouth, NS B5A 2J9, Canada   
3       11040 Santa Monica Blvd Suite 370, Los Angeles...   
4                                                     NaN   
...                                                   ...   
346920  12b Marsh Cl, Winterbourne, Bristol BS36 1HY, ...   
346921                                                NaN   
346922                                                NaN   
346923                                                NaN   
346924                   8024 22nd Ave, Kenosha, WI 53143   

                                 category            city country_code  \
0                 Fabric-Based Home Goods         gosford           au   
1                             Book Stores  st. catharines           ca   
2       Other B

In [99]:
website_data['clean_city'] = website_data['main_city'].apply(clean_address_component)
website_data['clean_region'] = website_data['main_region'].apply(clean_address_component)
website_data['clean_country'] = website_data['main_country'].apply(clean_address_component)

In [100]:
website_data.head

<bound method NDFrame.head of                       root_domain domain_suffix language  \
0                converterman.com           NaN      NaN   
1             clothesencounter.ca            ca       en   
2                  investa.com.au        com.au       en   
3               timminsgarage.com           com       en   
4                    destinate.ca            ca       en   
...                           ...           ...      ...   
72013                agistudio.ca            ca       en   
72014                    le279.ca            ca       fr   
72015  absolutedatadestruction.ca            ca       en   
72016                     hhms.ca            ca       en   
72017       studioaeinteriors.com           com       en   

                                       legal_name       main_city  \
0                                             NaN             NaN   
1                                             NaN        cardigan   
2      Investa Wholesale Funds Management 

In [101]:
# handling missing domains

In [102]:
facebook_no_domain = facebook_data[facebook_data['domain'].isnull()]

In [105]:
facebook_no_domain

,domain,address,categories,city,country_code,country_name,description,email,link,name,...,phone,phone_country_code,region_code,region_name,zip_code,clean_name,clean_phone,clean_city,clean_region,clean_country


In [103]:
google_no_domain = google_data[google_data['domain'].isnull()]

In [106]:
google_no_domain

,address,category,city,country_code,country_name,name,phone,phone_country_code,raw_address,raw_phone,region_code,region_name,text,zip_code,domain,clean_name,clean_phone,clean_city,clean_region,clean_country
8582,"Ufimskoye Shosse, 4, Ufa, Republic of Bashkort...",NaN,ufa,ru,russia,"Makheyev\"",+78003333693,ru,Ufa","Russia""",+7 800 333-36-93,ba,bashkortostan republic,"Food store Ufa, Russia Permanently closed · +7...",450000,maheev.ru,NaN,NaN,makheyev78003333693ruufa,,450000,food store ufa russia permanently closed 7 800...,ru
185343,"Ulitsa Zarubina, 150, Saratov, Saratov Oblast,...",NaN,saratov,ru,russia,"Firmennyy Ofis \Angstrem\""",",+78452291768,ru,"",+7 845 229-17-68,sar,sarato...",Russia · In Vash Dom · +7 845 229-17-68 Close...,410005,angstrem-mebel.ru,NaN,NaN,NaN,NaN,NaN,firmennyy ofis angstrem,7845229176878452291768,NaN,NaN,ru


In [104]:
website_no_domain = website_data[website_data['domain'].isnull()]

In [107]:
website_no_domain

,root_domain,domain_suffix,language,legal_name,main_city,main_country,main_region,phone,site_name,tld,s_category,domain,clean_legal_name,clean_site_name,clean_phone,clean_city,clean_region,clean_country


In [108]:
# matching on domain
fb_google_merged = pd.merge(facebook_data, google_data, on='domain', how='inner', suffixes=('_fb', '_gg'))

In [110]:
fb_google_merged

,domain,address_fb,categories,city_fb,country_code_fb,country_name_fb,description,email,link,name_fb,...,raw_phone,region_code_gg,region_name_gg,text,zip_code_gg,clean_name_gg,clean_phone_gg,clean_city_gg,clean_region_gg,clean_country_gg
0,euro-hygiene-34.fr,"134 rue entrepreneurs, za du vigné, 30420, cal...",NaN,calvisson,fr,france,NaN,NaN,https://euro-hygiene-34.fr,Euro Hygiène,...,+33 4 66 04 02 00,occ,occitanie,4.7 (3) · Pest control service 3+ years in bus...,NaN,euro hygiène,NaN,occitanie,occ,fr
1,lakesidehomeservices.ca,NaN,Appliance Repair & Maintenance|Home Builders &...,NaN,ca,NaN,NaN,NaN,https://lakesidehomeservices.ca,Lakeside Home Services,...,+1 613-831-8840,NaN,NaN,5.0 (2) · Appliance repair service Closed ⋅ Op...,NaN,lakeside home services,161383188400,NaN,NaN,ca
2,rossiterboats.com,NaN,Boats & Yachts Dealers|Boat Tours & Cruises,NaN,NaN,NaN,At Rossiter Boats we build semi-custom power b...,info@rossiterboats.com,http://rossiterboats.com,Rossiter Boats,...,+1 519-986-1203,on,ontario,"4.5 (4) · Boat builders Markdale, ON, Canada O...",NaN,rossiter boats,15199861203,ontario,on,ca
3,caa-architecture.com,"601 cordova st w # 270, v6b 1g1, vancouver, bc...",Architects & Architectural Services|Other Engi...,vancouver,ca,canada,NaN,office@caa-arcitecture.com,http://caa-architecture.com,Chandler Associates Architecture Inc.,...,+1 604-687-3390,bc,british columbia,4.5 (2) · Architect 5+ years in business · Van...,v6c 2g8,chandler associates architecture,16046873390,british columbia,bc,ca
4,apexsurety.ca,"unit 3 - 4 donald street, r3l 2t7, winnipeg, m...",NaN,winnipeg,ca,canada,NaN,NaN,https://apexsurety.ca,Apex Surety & Insurance Ltd.,...,+1 204-560-2508,mb,manitoba,No reviews · Insurance broker 4 Donald St #3 O...,r3l 2t7,apex surety insurance,120456025080,manitoba,mb,ca
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345258,cadex.com,"22000 fraserwood way, v6w1j6, richmond, bc, ca...",NaN,richmond,ca,canada,Cadex specializes in the design and manufactur...,answers@cadex.com,http://cadex.com,Cadex Electronics,...,+1 604-231-7777,bc,british columbia,"4.5 (4) · Electronics store Richmond, BC, Cana...",v6w 1j6,cadex electronics,160423177770,british columbia,bc,ca
345259,coastlinecatering.ca,"351 prince edward drive north, m8x 2l4, toront...",NaN,NaN,ca,NaN,NaN,NaN,https://coastlinecatering.ca,Coastline Catering,...,+1 416-294-9022,on,ontario,5.0 (10) · Caterer 7+ years in business · Etob...,m8x 2l4,coastline catering,14162949022,ontario,on,ca
345260,hcwd.com,"1400 rogersville rd, 40160, radcliff, ky, kent...",Utility companies|Auto Services|Utility companies,radcliff,us,united states,NaN,NaN,https://hcwd.com,Hardin County Water District No. 1,...,(270) 351-3222,ky,kentucky,"2.2 (16) · Water utility company Radcliff, KY ...",40160,hardin county water district no1,12703513222,kentucky,ky,us
345261,serenityskincarestudio.com,"15 westmount rd s, unit 301, n2r 1p1, kitchene...",Hair Removal Service|Spa & Wellness Centers,kitchener,ca,canada,NaN,NaN,https://serenityskincarestudio.com,Serenity Skin Care Studio Inc,...,+1 519-747-2256,on,ontario,New on Google No reviews · Medical spa 15 West...,n2l 2k2,serenity skin care studio laser and medispa,15197472256,ontario,on,ca


In [112]:
fb_google_merged.head

<bound method NDFrame.head of                             domain  \
0               euro-hygiene-34.fr   
1          lakesidehomeservices.ca   
2                rossiterboats.com   
3             caa-architecture.com   
4                    apexsurety.ca   
...                            ...   
345258                   cadex.com   
345259        coastlinecatering.ca   
345260                    hcwd.com   
345261  serenityskincarestudio.com   
345262     easttorontofootcare.com   

                                               address_fb  \
0       134 rue entrepreneurs, za du vigné, 30420, cal...   
1                                                     NaN   
2                                                     NaN   
3       601 cordova st w # 270, v6b 1g1, vancouver, bc...   
4       unit 3 - 4 donald street, r3l 2t7, winnipeg, m...   
...                                                   ...   
345258  22000 fraserwood way, v6w1j6, richmond, bc, ca...   
345259  351 prince edward

In [113]:
merged_data = pd.merge(fb_google_merged, website_data, on='domain', how='inner')

In [114]:
merged_data

,domain,address_fb,categories,city_fb,country_code_fb,country_name_fb,description,email,link,name_fb,...,phone,site_name,tld,s_category,clean_legal_name,clean_site_name,clean_phone,clean_city,clean_region,clean_country
0,minsa.gob.pe,"km. 2 salida huancane, 15082 perú, juliaca, pe...",NaN,juliaca,pe,peru,NaN,NaN,https://minsa.gob.pe,Red de Salud San Román,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,minsa.gob.pe,"km. 2 salida huancane, 15082 perú, juliaca, pe...",NaN,juliaca,pe,peru,NaN,NaN,https://minsa.gob.pe,Red de Salud San Román,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,minsa.gob.pe,"km. 2 salida huancane, 15082 perú, juliaca, pe...",NaN,juliaca,pe,peru,NaN,NaN,https://minsa.gob.pe,Red de Salud San Román,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,minsa.gob.pe,"km. 2 salida huancane, 15082 perú, juliaca, pe...",NaN,juliaca,pe,peru,NaN,NaN,https://minsa.gob.pe,Red de Salud San Román,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,royalmachinesolutions.ca,"1125 colborne street east, n3s 0h4, brantford,...",Business Consulting|Industrial Machinery & Sup...,brantford,ca,canada,NaN,NaN,https://royalmachinesolutions.ca,Royal Machine Solutions,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2152,mediatheque-paysdesommieres.com,"245 bd ernest françois, 30250, sommières, fran...",Libraries,sommieres,fr,france,NaN,NaN,https://mediatheque-paysdesommieres.com,Médiathèque Lawrence Durrell Sommières,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2153,mediatheque-paysdesommieres.com,"245 bd ernest françois, 30250, sommières, fran...",Libraries,sommieres,fr,france,NaN,NaN,https://mediatheque-paysdesommieres.com,Médiathèque Lawrence Durrell Sommières,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2154,mediatheque-paysdesommieres.com,"245 bd ernest françois, 30250, sommières, fran...",Libraries,sommieres,fr,france,NaN,NaN,https://mediatheque-paysdesommieres.com,Médiathèque Lawrence Durrell Sommières,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2155,mediatheque-paysdesommieres.com,"245 bd ernest françois, 30250, sommières, fran...",Libraries,sommieres,fr,france,NaN,NaN,https://mediatheque-paysdesommieres.com,Médiathèque Lawrence Durrell Sommières,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [115]:
merged_data.head

<bound method NDFrame.head of                                domain  \
0                        minsa.gob.pe   
1                        minsa.gob.pe   
2                        minsa.gob.pe   
3                        minsa.gob.pe   
4            royalmachinesolutions.ca   
...                               ...   
2152  mediatheque-paysdesommieres.com   
2153  mediatheque-paysdesommieres.com   
2154  mediatheque-paysdesommieres.com   
2155  mediatheque-paysdesommieres.com   
2156                  reefquarium.com   

                                             address_fb  \
0     km. 2 salida huancane, 15082 perú, juliaca, pe...   
1     km. 2 salida huancane, 15082 perú, juliaca, pe...   
2     km. 2 salida huancane, 15082 perú, juliaca, pe...   
3     km. 2 salida huancane, 15082 perú, juliaca, pe...   
4     1125 colborne street east, n3s 0h4, brantford,...   
...                                                 ...   
2152  245 bd ernest françois, 30250, sommières, fran...   
2153 

In [116]:
# records without domain

In [122]:
facebook_no_domain = facebook_no_domain[['clean_name', 'clean_phone', 'clean_city', 'clean_region', 'clean_country']]
google_no_domain = google_no_domain[['clean_name', 'clean_phone', 'clean_city', 'clean_region', 'clean_country']]
website_no_domain = website_no_domain[['clean_legal_name', 'clean_phone', 'clean_city', 'clean_region', 'clean_country']]

KeyError: "['clean_legal_name'] not in index"

In [123]:
website_no_domain = website_no_domain.rename(columns={'clean_legal_name': 'clean_name'}) # for consistency

In [134]:
# fuzzy matching - https://www.datacamp.com/tutorial/fuzzy-string-python
def fuzzy_match(name, choices, scorer, cutoff):
    if pd.isnull(name):
        return np.nan
    match = process.extractOne(name, choices, scorer=scorer)
    if match:
        matched_name, score = match
        if score >= cutoff:
            return matched_name
    return np.nan


In [135]:
website_names = website_no_domain['clean_name'].dropna().unique().tolist()

In [136]:
website_names

[]

In [137]:
facebook_no_domain['matched_name'] = facebook_no_domain.apply(
    lambda row: fuzzy_match(row['clean_name'], website_names, fuzz.token_sort_ratio, 85), axis=1)

In [138]:
# records where a match was found
facebook_fuzzy_matched = facebook_no_domain.dropna(subset=['matched_name'])

In [142]:
facebook_fuzzy_matched

,clean_name,clean_phone,clean_city,clean_region,clean_country,matched_name


In [146]:
facebook_fuzzy_matched = facebook_fuzzy_matched.dropna(subset=['matched_name'])

In [147]:
facebook_fuzzy_matched['matched_name'] = facebook_fuzzy_matched['matched_name'].astype(str)

In [148]:
website_no_domain['clean_name'] = website_no_domain['clean_name'].astype(str)

In [149]:
facebook_fuzzy_matched['matched_name'].dtype
website_no_domain['clean_name'].dtype

dtype('O')

In [150]:
facebook_fuzzy_merged = pd.merge(
    facebook_fuzzy_matched,
    website_no_domain,
    left_on='matched_name',
    right_on='clean_name',
    how='inner',
    suffixes=('_fb', '_ws')
)

In [153]:
facebook_fuzzy_merged

,clean_name_fb,clean_phone_fb,clean_city_fb,clean_region_fb,clean_country_fb,matched_name,clean_name_ws,clean_phone_ws,clean_city_ws,clean_region_ws,clean_country_ws


In [253]:
final_merged_data = pd.concat([merged_data, facebook_fuzzy_merged], ignore_index=True)

In [254]:
# rename for consistency
final_merged_data = final_merged_data.rename(columns={
    'legal_name': 'name_ws',
    'name_fb': 'name_fb',
    'name_gg': 'name_gg',
    'site_name': 'name_ws',
    'region_name_fb': 'region_fb',
    'region_code_gg': 'region_gg',
    'main_region_ws': 'region_ws',
    'country_code_fb': 'country_fb',
    'country_code_gg': 'country_gg',
    'main_country_ws': 'country_ws',
    
})

In [255]:
# conflict resolution function

In [256]:
# removing duplicates
duplicate_columns = final_merged_data.columns[final_merged_data.columns.duplicated()].unique()

final_merged_data = final_merged_data.loc[:, ~final_merged_data.columns.duplicated()]


In [257]:
expected_columns = []
for col in columns_to_resolve:
    for source in priority_order:
        expected_columns.append(f'{col}_{source}')

In [258]:
def resolve_conflict(row, col, priority_order):
    for source in priority_order:
        col_name = f'{col}_{source}'
        if col_name in row.index:
            value = row[col_name]
            if isinstance(value, pd.Series):
                # Handle duplicate columns by selecting the first non-null value
                value = value.dropna().iloc[0] if not value.dropna().empty else np.nan
            if pd.notnull(value):
                return value
    return np.nan


In [259]:
columns_to_resolve = ['name', 'phone', 'address', 'city', 'region', 'country']

# defining the priority 
priority_order = ['ws', 'gg', 'fb'] 


In [260]:
for col in columns_to_resolve:
    final_merged_data[col] = final_merged_data.apply(
        lambda row: resolve_conflict(row, col, priority_order), axis=1)

In [261]:
# source-specific columns
columns_to_drop = [col for col in final_merged_data.columns if any(col.endswith(f'_{suffix}') for suffix in ['fb', 'gg', 'ws'])]
final_merged_data = final_merged_data.drop(columns=columns_to_drop)

In [262]:
desired_columns = ['domain', 'name', 'phone', 'email', 'address', 'city', 'region', 'country', 'zip_code', 'category', 'description']

In [263]:
# filtering & reordering the data frame
final_columns = [col for col in desired_columns if col in final_merged_data.columns]

final_merged_data = final_merged_data[final_columns]

In [264]:
# dropping records with missing data
final_merged_data = final_merged_data.dropna(subset=['domain', 'name'])


In [265]:
final_merged_data.to_csv('FINAL.csv', index=False)
